In [1]:
%config IPCompleter.use_jedi = False
%pdb off
# %load_ext viztracer
# from viztracer import VizTracer
%load_ext autoreload
%autoreload 3
import sys
import subprocess
import inspect
import pathlib
from pathlib import Path
import inspect
from functools import wraps
import dill

from pyphocorehelpers.Filesystem.source_code_helpers import find_py_files
from pyphocorehelpers.Filesystem.generate_code_index import build_code_index

Automatic pdb calling has been turned OFF


In [4]:
# Import my libraries:
import neuropy
import neuropy.core

In [5]:

import pyphocorehelpers as pho_help
import pyphoplacecellanalysis as pho_pca


In [6]:
def find_functions(module):
    return [name for name, obj in inspect.getmembers(module) if inspect.isfunction(obj)]

## Find all functions in `pho_help`
# found_functions = find_functions(module=pho_help)


In [7]:
# module = pho_help
# module = pho_pca
module = neuropy.core

found_any =  [name for name, obj in inspect.getmembers(module)]
print(found_any)

['Animal', 'BinnedSpiketrain', 'DataWriter', 'Epoch', 'FlattenedSpiketrains', 'Laps', 'Mua', 'NeuronType', 'Neurons', 'Position', 'Probe', 'ProbeGroup', 'Ratemap', 'Shank', 'Signal', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'animal', 'datawriter', 'epoch', 'flattened_spiketrains', 'laps', 'neuron_identities', 'neurons', 'position', 'probe', 'ratemap', 'signal']


Discovering modules: 

https://stackoverflow.com/questions/4858100/how-to-list-imported-modules

In [8]:
from modulefinder import ModuleFinder
finder = ModuleFinder()
finder.run_script(r"C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\scripts\test_generate_icon_with_overlay.py")
for name, mod in finder.modules.items():
    print(name)
    

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\pho\\repos\\Spike3DWorkEnv\\Spike3D\\scripts\\test_generate_icon_with_overlay.py'

In [7]:
pho_help

<module 'pyphocorehelpers' from 'C:\\Users\\pho\\repos\\Spike3DWorkEnv\\pyPhoCoreHelpers\\src\\pyphocorehelpers\\__init__.py'>

# 2023-09-25 Code Parsing

In [8]:
import dill



In [ ]:
dill.source.

get_objgraph ArrayType


In [9]:


# def enumerate_classes_and_functions(module):
#     items = {}
#     for name, obj in inspect.getmembers(module):
#         if inspect.ismodule(obj):
#             items[name] = enumerate_classes_and_functions(obj)
#         elif inspect.isclass(obj) or inspect.isfunction(obj):
#             items[name] = str(inspect.getdoc(obj))
#     return items

# # Example usage with dill library as target module
# result = enumerate_classes_and_functions(dill)
# print(result)



# Function to find all functions with a certain decorator
def find_functions_with_decorator(module, decorator):
    result = []
    for name, obj in inspect.getmembers(module):
        if inspect.isfunction(obj) and hasattr(obj, '__wrapped__'):
            if obj.__wrapped__.__module__ == decorator.__module__:
                result.append(name)
        elif inspect.ismodule(obj) or inspect.isclass(obj):
            result.extend(find_functions_with_decorator(obj, decorator))
    return result


found_functions = find_functions_with_decorator(__import__(pho_help), my_decorator)
print(found_functions)

TypeError: __import__() argument 1 must be str, not module

## 2023-09-21 - `dill` Checking

In [ ]:
# dill.errors(obj, depth=0, exact=False, safe=False) # get errors for objects that fail to pickle
# dill.pickles(obj, exact=False, safe=False, **kwds) # Quick check if object pickles with dill.
# dill.get_objgraph

def dump_session(file_path):
  """Pickle the current python session to be used in the worker.

  Note: Due to the inconsistency in the first dump of dill dump_session we
  create and load the dump twice to have consistent results in the worker and
  the running session. Check: https://github.com/uqfoundation/dill/issues/195
  """
  dill.dump_session(file_path)
  dill.load_session(file_path)
  return dill.dump_session(file_path)

test_session_file = Path('../output/test_pickled_session_2023-09-21.pkl').resolve()
print(f'saving session to {test_session_file}')
dump_session(test_session_file)

# dill.detect.trace(True)
# dill.dump_session(test_session_file) # PicklingError: Can't pickle : it's not the same object as builtins.input
# PicklingError: Can't pickle .NewCol'>: it's not found as tables.description.Col._subclass_from_prefix..NewCol

# with detect.trace('output/dill_trace_2023-09-21.log', mode='w') as log:
#     # log("> D = %r", D)
#     dill.dump_session(test_session_file)


In [ ]:

dill.detect.trace(False)
# dump_session(test_session_file)


In [ ]:
 
dill.dump_session()

# Pre 2023-09-25

In [11]:
import subprocess
import pathlib

def build_code_index(project_path):
    project_path = pathlib.Path(project_path)
    cscope_files = project_path / "cscope.files"
    cscope_out = project_path / "cscope.out"
    tags = project_path / "tags"
    db_path = project_path / "myproject.db"
    
    # Generate the cscope index
    subprocess.run(["pycscope", "-i", str(cscope_files)], cwd=str(project_path), check=True)
    
    # Generate the ctags index
    subprocess.run(["ctags", "--fields=+i", "-n", "-L", str(cscope_files)], cwd=str(project_path), check=True)
    
    # Generate the CodeQL database
    subprocess.run(["cqmakedb", "-s", str(db_path), "-c", str(cscope_out), "-t", str(tags), "-p"], cwd=str(project_path), check=True)


build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src")

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [10]:
# project_outputs = build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy", project_name='NeuroPy')
# project_path, project_db_file_path, tags_file_path, cscope_out_file_path, cscope_files_file_path = project_outputs # unpack

project_outputs = build_code_index(r"C:\Users\pho\repos\Spike3DWorkEnv\NeuroPy\neuropy", project_name='NeuroPy')


AssertionError: 

In [ ]:
project_outputs = build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src", project_name='pyPhoCoreHelpers')

In [ ]:
project_outputs = build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"], project_name='pyPhoPlaceCellAnalysis')

In [12]:
import glob
import importlib
import inspect
import os

current_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)))
current_module_name = os.path.splitext(os.path.basename(current_dir))[0]
for file in glob.glob(current_dir + "/*.py"):
     name = os.path.splitext(os.path.basename(file))[0]

     # Ignore __ files
     if name.startswith("__"):
         continue
     module = importlib.import_module("." + name,package=current_module_name)

     for member in dir(module):
         handler_class = getattr(module, member)

         if handler_class and inspect.isclass(handler_class):
             print(member)
             


NameError: name '__file__' is not defined

In [ ]:


classes = [cls_name for cls_name, cls_obj in inspect.getmembers(sys.modules['test_cls']) if inspect.isclass(cls_obj)]
print(classes)


# Pre 2023-07-28 Testing

In [11]:
root_project_path = pathlib.Path("C:/Users/pho/repos/Spike3DWorkEnv/")
cscope_files = root_project_path / "cscope.files"
cscope_out = root_project_path / "cscope.out"
tags = root_project_path / "tags"
db_path = root_project_path / "myproject.db"

repos_source_paths = ["C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy", "C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src", "C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src"]
repos_exclude_dir_lists = [None, None, ["pyphoplacecellanalysis/External"]]

all_found_files = []
for a_src_path, exclude_dirs in zip(repos_source_paths, repos_exclude_dir_lists):
    all_found_files.extend(find_py_files(a_src_path, exclude_dirs = exclude_dirs))

all_found_files

# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy")
# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src")
# find_py_files("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"])



[WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/__init__.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/.ipynb_checkpoints/__init__-checkpoint.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/artifact.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/brainstates.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/decoders.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/laps.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/ms_connectivity.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/neurons_correlation.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/neurons_stability.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/oscillations.py'),
 WindowsPath('C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy/analyses/place

In [ ]:
# Write the file paths to cscope.files
with open(cscope_files, "w") as f:
	print(f'writing to {cscope_files}...')
	for file_path in all_found_files:
		f.write(str(file_path) + "\n")
	print('\tdone.')

# Old Full Implementation

In [ ]:

def build_code_index(project_path, exclude_dirs=[]):
    project_path = pathlib.Path(project_path)
    cscope_files = project_path / "cscope.files"
    cscope_out = project_path / "cscope.out"
    tags = project_path / "tags"
    db_path = project_path / "myproject.db"

    # Find all .py files in the project directory and its subdirectories
    py_files = project_path.glob("**/*.py")
    py_files = [file_path for file_path in py_files] # to list

    excluded_py_files = []
    if exclude_dirs is not None:
        # Find all .py files in the project directory and its subdirectories, excluding the 'my_exclude_dir' directory
        exclude_paths = [project_path.joinpath(a_dir) for a_dir in exclude_dirs]
        for an_exclude_path in exclude_paths:
            excluded_py_files.extend([file_path for file_path in an_exclude_path.glob("**/*.py")])

    included_py_files = [x for x in py_files if x not in excluded_py_files]

    # Write the file paths to cscope.files
    with open(project_path / "cscope.files", "w") as f:
        for file_path in included_py_files:
            f.write(str(file_path) + "\n")

    # Generate the cscope index
    subprocess.run(["pycscope", "-i", str(cscope_files)], cwd=str(project_path), check=True)
    
    # Generate the ctags index
    subprocess.run(["ctags", "--fields=+i", "-n", "-L", str(cscope_files)], cwd=str(project_path), check=True)
    
    # Generate the CodeQL database
    subprocess.run(["cqmakedb", "-s", str(db_path), "-c", str(cscope_out), "-t", str(tags), "-p"], cwd=str(project_path), check=True)



In [ ]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/NeuroPy/neuropy")

In [ ]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoCoreHelpers/src")

In [ ]:
build_code_index("C:/Users/pho/repos/Spike3DWorkEnv/pyPhoPlaceCellAnalysis/src", exclude_dirs = ["pyphoplacecellanalysis/External"])